In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'../library')

In [2]:
import vexa
from dotenv import load_dotenv
import os
load_dotenv()
STREAM_API_PORT = os.getenv('STREAM_API_PORT')
DATA_PATH = os.getenv('DATA_PATH')
SERVICE_TOKEN = os.getenv('SERVICE_TOKEN')

In [3]:
from vexa.redis import get_redis

In [4]:
redis_client = await get_redis(host='redis',port=6379)

In [7]:
import httpx

In [8]:
async def get_connections():

    url = f"http://host.docker.internal:{STREAM_API_PORT}/list_connections"
    params = {
            "service_token": SERVICE_TOKEN
        }
    headers = {
        "Content-Type": "application/json"
    }


    async with httpx.AsyncClient() as client:
            response = await client.get(url, params=params, headers=headers, timeout=2)
    
    return response.json()['connections']



In [31]:
async def get_next_audio_chunk(connection_id):
    #TODO: add error handlingdepending in case 
    url = f"http://host.docker.internal:{STREAM_API_PORT}/get_next_chunk/{connection_id}"
    headers = {
            "Content-Type": "application/json"
        }
    params = {
                "service_token": SERVICE_TOKEN
            }


    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params, headers=headers, timeout=2)
        if response.json() == {'message': 'No more chunks available for this connection'}:
            return 
    return response.json()
    

In [52]:
async def writestream2file(conn_id):
    path = f'/audio/{conn_id}.webm'
    item = True
    while item:
        item = await  get_next_audio_chunk(conn_id)
        if item:
            chunk = bytes.fromhex(item['chunk']['chunk'])
            # Open the file in append mode
            with open(path, 'ab') as file:
                # Write data to the file
                file.write(chunk)
            last_timestamp = item['chunk']['timestamp']
            meeting_id = item['chunk']['meeting_id']
            client_id = item['chunk']['client_id']
    return meeting_id, last_timestamp,client_id

In [72]:
connections = await get_connections()

In [73]:
connections = [c[0] for c in connections]

In [74]:
connections

['ece9a33c-d069-4f3b-b7c3-6c0d29d08db7--0']

In [95]:
connection_id = connections[0]

In [96]:
meeting_id, timestamp, client_id = await  writestream2file(connection_id)

In [97]:
import pandas as pd

In [98]:
pd.Timestamp(timestamp)

Timestamp('2024-05-03 20:32:12.414516+0000', tz='UTC')

In [99]:
pd.Timestamp.utcnow()

Timestamp('2024-05-03 20:32:16.281421+0000', tz='UTC')

In [178]:
async def meeting_start(timestamp):
    meeting_start = await redis_client.hget(f'Meeting:{meeting_id}','meeting_start')
    meeting_start = meeting_start if meeting_start else timestamp
    await redis_client.hset(f'Meeting:{meeting_id}','meeting_start',timestamp)
    return pd.Timestamp(meeting_start)

In [ ]:
async def last_start(timestamp):
    last_start = await redis_client.hget(f'Meeting:{meeting_id}','last_start')
    last_start = last_start if last_start else timestamp
    await redis_client.hset(f'Meeting:{meeting_id}','last_start',timestamp)
    return pd.Timestamp(meeting_start)

In [177]:
await meeting_start(timestamp)

Timestamp('2024-05-03 20:32:12.414516+0000', tz='UTC')

In [103]:
start = await redis_client.rpop(f'Start:{meeting_id}')
start = float(start) if start else pd.Timestamp(timestamp)

In [132]:
step = pd.Timedelta('1 min')

In [133]:
(pd.Timestamp.utcnow()-start)>step

True

In [135]:
from audio.audio import *

In [139]:
max_length = 200

In [140]:
path = f'/audio/{connection_id}.webm'

In [181]:
meeting_start = await meeting_start(timestamp)

In [141]:
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,start+max_length)

TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`